# 3.6 Implementation of Softmax Regression from Scratch
Just as we implemented linear regression from scratch, we believe that softmax regression is similarly fundamental and you ought to know the gory details of
how to implement it yourself. We will work with the Fashion-MNIST dataset, just introduced in
Section 3.5, setting up a data iterator with batch size 256.

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use AI::MXNet::Gluon qw(gluon);
use d2l;
use d2l::Accumulator;
use d2l::Animator;
use List::Util qw(min max shuffle);
use Scalar::Util qw(blessed );
IPerl->load_plugin('Chart::Plotly');
use Cwd 'abs_path';

In [2]:
my $batch_size = 256;

my $transform = sub {my ($data, $label) = @_;                 
                       return ($data->reshape([-1, 28, 28])->astype('float32') / 255, $label)};

CODE(0xacbd2a0)

In [3]:
my $data_train = gluon->data->vision->FashionMNIST('~/.mxnet/datasets/fashion-mnist', train => 1);
my $data_test  = gluon->data->vision->FashionMNIST('~/.mxnet/datasets/fashion-mnist',  train => 0);

AI::MXNet::Gluon::Data::Vision::DownloadedDataSet::FashionMNIST=HASH(0xacc4c90)

In [4]:
my ($X_train, $y_train) = $transform->($data_train->{data}, mx->nd->array($data_train->{label}));
my ($X_test, $y_test)   = $transform->($data_test->{data} , mx->nd->array($data_test->{label}));

<AI::MXNet::NDArray 10000x28x28 @cpu(0)><AI::MXNet::NDArray 10000 @cpu(0)>

In [5]:
sub data_iter_sequential{ # Optimized for sequential minibatches
  my ($features, $labels, $batch_size) = @_;
  my $num_samples = $features->len;
  my @indices = (0 .. $num_samples - 1);
  my ($index, @batch_indices) = 0;

  return sub {
    if (defined $_[0] && $_[0] == 0){# Reset
      $index = 0;
      return 1;
    }
    return undef if ($index >= $num_samples);
    @batch_indices = @indices[$index .. min($index + $batch_size, $num_samples) - 1];
    $index += $batch_size;
    return {data  => $features->slice([$batch_indices[0], $batch_indices[-1]]), 
            label => $labels->slice([$batch_indices[0], $batch_indices[-1]])};
  };
}

In [6]:
my $train_iter = data_iter_sequential($X_train, $y_train, $batch_size);
my $test_iter  = data_iter_sequential($X_test, $y_test, $batch_size);

CODE(0xb4a1710)

We use the Fashion-MNIST data set with batch size 256.

## 3.6.1 Initializing Model Parameters
As in our linear regression example, each example here will be represented by a fixed-length vector. Each example in the raw dataset is a 28×28 image. In this section, we will flatten each image,
treating them as vectors of length 784. In the future, we will talk about more sophisticated strategies for exploiting the spatial structure in images, but for now we treat each pixel location as just
another feature.

Recall that in softmax regression, we have as many outputs as there are classes. Because our
dataset has 10 classes, our network will have an output dimension of 10. Consequently, our weights will constitute a 784 × 10 matrix and the biases will constitute a 1 × 10 row vector. As with linear
regression, we will initialize our weights W with Gaussian noise and our biases to take the initial
value 0.

In [7]:
##num_inputs = 784
##num_outputs = 10

##W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
## b = nd.zeros(num_outputs)
##W.attach_grad()
##b.attach_grad()

my $num_inputs = 784;
my $num_outputs = 10;

our $W = mx->nd->random_normal(0, 0.01, [$num_inputs,$num_outputs]);
our $b = mx->nd->zeros([$num_outputs]);

$W->attach_grad();
$b->attach_grad();

## 3.6.2 Defining the Softmax Operation
Before implementing the softmax regression model, let us briefly review how the sum operator
works along specific dimensions in a tensor, as discussed in Section 2.3.6 and Section 2.3.6. Given
a matrix X we can sum over all elements (by default) or only over elements in the same axis, i.e.,
the same column (axis 0) or the same row (axis 1). Note that if X is a tensor with shape (2, 3) and we
sum over the columns, the result will be a vector with shape (3,). When invoking the sum operator,
we can specify to keep the number of axes in the original tensor, rather than collapsing out the
dimension that we summed over. This will result in a two-dimensional tensor with shape (1, 3).

In [8]:
##X = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
##X.sum(0, keepdims=True), X.sum(1, keepdims=True)
my $X = mx->nd->array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]);
my $sum_X = $X->sum(axis => 0, keepdims => 1), $X->sum(axis => 1, keepdims => 1);
print "\n", dump $sum_X->asarray;


[[5, 7, 9]]

1

We are now ready to implement the softmax operation. Recall that softmax consists of three steps:
i) we exponentiate each term (using exp); ii) we sum over each row (we have one row per example
in the batch) to get the normalization constant for each example; iii) we divide each row by its
normalization constant, ensuring that the result sums to 1. Before looking at the code, let us
recall how this looks expressed as an equation:
$$softmax(X)_{ij} = \frac{exp(X_{ij})}{∑_kexp(X_{ik})} .  (3.6.1)$$ 

The denominator, or normalization constant, is also sometimes called the partition function (and
its logarithm is called the log-partition function). The origins of that name are in statistical
physics57 where a related equation models the distribution over an ensemble of particles.

In [9]:
##def softmax(X):
  ##  X_exp = np.exp(X)
    ##partition = X_exp.sum(1, keepdims=True)
    ##return X_exp / partition # The broadcasting mechanism is applied here
sub softmax{
    my ($X) = @_;
    my $X_exp = mx->nd->exp($X);
    my $partition = $X_exp->sum(axis => 1, keepdims => 1); 
    return $X_exp / $partition; # The broadcasting mechanism is applied here
}

As you can see, for any random input, we turn each element into a non-negative number. Moreover, each row sums up to 1, as is required for a probability.


In [10]:
##X = np.random.normal(0, 1, (2, 5))
##X_prob = softmax(X)
##X_prob, X_prob.sum(1)
$X = mx->nd->random_normal(0, 1, shape=>[2, 5]);
my $X_prob = softmax($X);
print dump $X_prob->sum(1)->asarray;

[1.00000011920929, 1]

1

Note that while this looks correct mathematically, we were a bit sloppy in our implementation
because we failed to take precautions against numerical overflow or underflow due to large or
very small elements of the matrix.
## 3.6.3 Defining the Model
Now that we have defined the softmax operation, we can implement the softmax regression model.
The below code defines how the input is mapped to the output through the network. Note that we
flatten each original image in the batch into a vector using the reshape function before passing
the data through our model.

In [11]:
##def net(X):
  ##  return softmax(nd.dot(X.reshape((-1, num_inputs)), W) + b)

sub net{
    my ($X) = @_;
    our $W;
    return softmax(mx->nd->dot($X->reshape([-1, $W->shape->[0]]), $W) + $b);
}


## 3.6.4 Defining the Loss Function
Next, we need to implement the cross-entropy loss function, as introduced in Section 3.4. This
may be the most common loss function in all of deep learning because, at the moment, classification problems far outnumber regression problems.

Recall that cross-entropy takes the negative log-likelihood of the predicted probability assigned to
the true label. Rather than iterating over the predictions with a Python for-loop (which tends to
be inefficient), we can pick all elements by a single operator. Below, we create sample data y_hat
with 2 examples of predicted probabilities over 3 classes and their corresponding labels y. With y
we know that in the first example the first class is the correct prediction and in the second example
the third class is the ground-truth. Using y as the indices of the probabilities in y_hat, we pick the
probability of the first class in the first example and the probability of the third class in the second
example.


In [12]:
##y = np.array([0, 2])
##y_hat = np.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
##y_hat[[0, 1], y]
my $y = mx->nd->array([0, 2]);
my $y_hat = mx->nd->array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]]);
my $aux = mx->nd->array([0,1]);
print dump $y_hat->slice([$aux, $y])->asarray;

[0.100000001490116, 0.5]

1

Now we can implement the cross-entropy loss function efficiently with just one line of code.

In [13]:
##def cross_entropy(y_hat, y):
  ##  return -np.log(y_hat[range(len(y_hat)), y])

##cross_entropy(y_hat, y)
sub cross_entropy{
    my ($y_hat, $y) = @_;
    my $len = $y_hat->len - 1;
    return -(mx->nd->log($y_hat->slice([0, $len])));
}
cross_entropy($y_hat, $y);

<AI::MXNet::NDArray 2x3 @cpu(0)>

## 3.6.5 Classification Accuracy
Given the predicted probability distribution y_hat, we typically choose the class with the highest
predicted probability whenever we must output a hard prediction. Indeed, many applications
require that we make a choice. Gmail must categorize an email into “Primary”, “Social”, “Updates”,
or “Forums”. It might estimate probabilities internally, but at the end of the day it has to choose
one among the classes.

When predictions are consistent with the label class y, they are correct. The classification accuracy is the fraction of all predictions that are correct. Although it can be difficult to optimize
accuracy directly (it is not differentiable), it is often the performance measure that we care most
about, and we will nearly always report it when training classifiers.

To compute accuracy we do the following. First, if y_hat is a matrix, we assume that the second
dimension stores prediction scores for each class. We use argmax to obtain the predicted class by
the index for the largest entry in each row. Then we compare the predicted class with the groundtruth y elementwise. Since the equality operator == is sensitive to data types, we convert y_hatʼs
data type to match that of y. The result is a tensor containing entries of 0 (false) and 1 (true).
Taking the sum yields the number of correct predictions.

In [14]:
##def accuracy(y_hat, y): #@save
  ##  """Compute the number of correct predictions."""
    ##if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
      ##  y_hat = y_hat.argmax(axis=1)
    ##cmp = y_hat.astype(y.dtype) == y
    ##return float(cmp.astype(y.dtype).sum())
sub accuracy{ #@save
    #Compute the number of correct predictions."""
    my($y_hat, $y) =@_;
    
    if (scalar @{$y_hat->shape} > 1 && $y_hat->shape->[1] > 1){
        $y_hat = $y_hat->argmax(axis => 1);
    }
        
    my $cmp = ($y_hat->astype($y->dtype) == $y);
    return $cmp->sum->asscalar;
}

    
    

We will continue to use the variables y_hat and y defined before as the predicted probability distributions and labels, respectively. We can see that the first exampleʼs prediction class is 2 (the
largest element of the row is 0.6 with the index 2), which is inconsistent with the actual label, 0.
The second exampleʼs prediction class is 2 (the largest element of the row is 0.5 with the index of
2), which is consistent with the actual label, 2. Therefore, the classification accuracy rate for these
two examples is 0.5.

In [15]:
##accuracy(y_hat, y) / len(y)
print accuracy($y_hat, $y) / $y->len;

0.5

1

Similarly, we can evaluate the accuracy for any model net on a dataset that is accessed via the data
iterator data_iter.

In [16]:
sub evaluate_accuracy{
    my ($net, $data_iter) = @_;#@save
    ##"""Compute the accuracy for a model on a dataset."""
    my $metric = Accumulator->new(2); # No. of correct predictions, no. of predictions
    my ($X, $y, $accuracy);
    while (defined(my $batch = $data_iter->())){
        $X = $batch->{data};
        $y = $batch->{label}->astype('float32');
        $accuracy = accuracy($net->($X), $y);
        $metric->add([ $accuracy, $y->size]);
    }
    print "Ninguna predición:", $metric->getitem(0);
    return ($metric->getitem(0) / $metric->getitem(1));
} 

Here Accumulator is a utility class to accumulate sums over multiple variables. In the above evaluate_accuracy function, we create 2 variables in the Accumulator instance for storing both the
number of correct predictions and the number of predictions, respectively. Both will be accumulated over time as we iterate over the dataset.

Because we initialized the net model with random weights, the accuracy of this model should be
close to random guessing, i.e., 0.1 for 10 classes.

In [17]:
##evaluate_accuracy(net, test_iter)
evaluate_accuracy(\&net, $test_iter);

Ninguna predición:925

0.0925

## 3.6.6 Training
The training loop for softmax regression should look strikingly familiar if you read through our
implementation of linear regression in Section 3.2. Here we refactor the implementation to make
it reusable. First, we define a function to train for one epoch. Note that updater is a general
function to update the model parameters, which accepts the batch size as an argument. It can be
either a wrapper of the d2l.sgd function or a frameworkʼs built-in optimization function.

In [18]:
sub train_epoch_ch3{ #@save
    my ($net, $train_iter, $loss, $updater) = @_;
    my $metric = Accumulator->new(3);
    if ($updater->isa('AI::MXNet::Gluon::Trainer') == 1){
        $updater = $updater->step
    }
    
    my ($X, $y, $y_hat, $accuracy, $l);
    while (defined(my $batch = $train_iter->())){
        $X = $batch->{data};
        $y = $batch->{label}->astype('float32');
    
        mx->autograd->record(sub {
            $y_hat = $net->($X);
            $l = cross_entropy($y_hat, $y);
            $l->backward;
        });
        
        $updater->($X->shape->[0]);
        $accuracy = accuracy($y_hat, $y);
        $metric->add([$l->sum()->astype('float32'), $accuracy, $y->size])
    }

    return ($metric->getitem(0) / $metric->getitem(2), $metric->getitem(1) / $metric->getitem(2));
}

Before showing the implementation of the training function, we define a utility class that plot data
in animation. Again, it aims to simplify code in the rest of the book.

The following training function then trains a model net on a training dataset accessed via
train_iter for multiple epochs, which is specified by num_epochs. At the end of each epoch, the
model is evaluated on a testing dataset accessed via test_iter. We will leverage the Animator class
to visualize the training progress.

In [19]:
##def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater): #@save
  ##  """Train a model (defined in Chapter 3)."""
    ##animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
     ##           legend=['train loss', 'train acc', 'test acc'])
   ## for epoch in range(num_epochs):
     ##   train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
       ## test_acc = evaluate_accuracy(net, test_iter)
        ##animator.add(epoch + 1, train_metrics + (test_acc,))
  ##  train_loss, train_acc = train_metrics
    
    ##assert train_loss < 0.5, train_loss
    ##assert train_acc <= 1 and train_acc > 0.7, train_acc
    ##assert test_acc <= 1 and test_acc > 0.7, test_acc

sub assertFunc{
    my ($cond, $print) = @_;
    if ($cond != 1){
        print $print."\n";
    }
}

sub train_ch3{ #@save
    my ($net, $train_iter, $test_iter, $loss, $num_epochs, $updater) = @_;
    my $animator = Animator(xlabel=>'epoch', xlim=>[1, $num_epochs], ylim=>[0.3, 0.9], legend=>['train loss', 'train acc', 'test acc']);
    
    my ($train_metrics, $test_acc);
    for my $epoch (0..$num_epochs){
        $train_metrics = train_epoch_ch3(\&net, $train_iter, $loss, $updater);
        $test_acc = evaluate_accuracy(\&net, $test_iter);
        $animator->add($epoch + 1, $train_metrics + ($test_acc,));
    }  
    
    my ($train_loss, $train_acc) = $train_metrics;
    
    assertFunc($train_loss < 0.5, $train_loss);
    assertFunc($train_acc <= 1 and $train_acc > 0.7, $train_acc);
    assertFunc($test_acc <= 1 and $test_acc > 0.7, $test_acc);
}

As an implementation from scratch, we use the minibatch stochastic gradient descent defined in
Section 3.2 to optimize the loss function of the model with a learning rate 0.1.

In [20]:
##lr = 0.1

##def updater(batch_size):
  ##  return d2l.sgd([W, b], lr, batch_size)
    
our $lr = 0.1;

sub updater{
    our $W;
    my ($batch_size) = @_;
    return d2l->sgd([$W, $b], learning_rate=>$lr, $batch_size);
}

0.1

Now we train the model with 10 epochs. Note that both the number of epochs (num_epochs), and
learning rate (lr) are adjustable hyperparameters. By changing their values, we may be able to
increase the classification accuracy of the model.

In [ ]:
##num_epochs = 10
##train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)
my $num_epochs = 10;
d2l->train_ch3(\&net, $train_iter, $test_iter, \&cross_entropy, $num_epochs, \&updater);

## 3.6.7 Prediction
Now that training is complete, our model is ready to classify some images. Given a series of images, we will compare their actual labels (first line of text output) and the predictions from the
model (second line of text output).

In [ ]:
sub predict_ch3{#@save
     #   """Predict labels (defined in Chapter 3)."""
    my ($net, $test_iter, $n) = @_; 
    $n = 6;
    my ($y, $X);
    my $trues = d2l->get_fashion_mnist_labels($y);
    my $preds = d2l->get_fashion_mnist_labels(net($X)->argmax(axis => 1));
    d2l->show_images(imgs => $X,
                        num_rows => 2,
                        num_cols => 9,
                        titles => $trues);
}

In [ ]:
d2l->predict_ch3(\&net, $test_iter);

## Summary
* With softmax regression, we can train models for multiclass classification.
* The training loop of softmax regression is very similar to that in linear regression: retrieve
and read data, define models and loss functions, then train models using optimization algorithms. As you will soon find out, most common deep learning models have similar training
procedures.